In [2]:

from pyspark.sql.session import SparkSession
from pyspark.sql.functions import col, dense_rank
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

teams = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", "teams") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

teams.show()

+---------+-------+-----------------+--------------+----+-----+------+---------+-------------+
|season_id|team_id|        team_name|matches_played|wins|draws|losses|goals_for|goals_against|
+---------+-------+-----------------+--------------+----+-----+------+---------+-------------+
|     2021|      1|  Manchester City|            38|  29|    6|     3|       99|           26|
|     2021|      2|        Liverpool|            38|  28|    8|     2|       94|           26|
|     2021|      3|          Chelsea|            38|  21|   11|     6|       76|           33|
|     2021|      4|        Tottenham|            38|  22|    5|    11|       69|           40|
|     2021|      5|          Arsenal|            38|  22|    3|    13|       61|           48|
|     2022|      1|  Manchester City|            38|  28|    5|     5|       94|           33|
|     2022|      2|          Arsenal|            38|  26|    6|     6|       88|           43|
|     2022|      3|Manchester United|            3

In [5]:
teams\
    .withColumn("points",col("wins")*3 + col("draws")*1 + col("losses")*0)\
    .withColumn("goal_difference",col("goals_for")-col("goals_against"))\
    .withColumn("drnk",dense_rank().over(Window.partitionBy(col("season_id")).orderBy(col("points").desc())))\
    .drop("wins","losses","goals_for","goals_against","matches_played","draws")\
    .show()

+---------+-------+-----------------+------+---------------+----+
|season_id|team_id|        team_name|points|goal_difference|drnk|
+---------+-------+-----------------+------+---------------+----+
|     2021|      1|  Manchester City|    93|             73|   1|
|     2021|      2|        Liverpool|    92|             68|   2|
|     2021|      3|          Chelsea|    74|             43|   3|
|     2021|      4|        Tottenham|    71|             29|   4|
|     2021|      5|          Arsenal|    69|             13|   5|
|     2022|      1|  Manchester City|    89|             61|   1|
|     2022|      2|          Arsenal|    84|             45|   2|
|     2022|      3|Manchester United|    75|             15|   3|
|     2022|      4|        Newcastle|    71|             35|   4|
|     2022|      5|        Liverpool|    67|             28|   5|
+---------+-------+-----------------+------+---------------+----+

